In [10]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load the data
df = pd.read_excel("Alphabet_cashflow1.xlsx", index_col=0)

# Function to create a forecasting DataFrame from the original DataFrame
def for_forecast_df_from_orig_df(orig_df, series_name):
    for_fcst_df = pd.DataFrame(orig_df.loc[series_name])
    for_fcst_df = for_fcst_df.reset_index(drop=True).reset_index()
    for_fcst_df = for_fcst_df.rename(columns={'index': 't'})
    return for_fcst_df

# Function to compute intercept and beta from forecasting DataFrame
def intercept_and_t_beta_from_for_forecast_df(for_fcst_df, series_name):
    # Ensure the data is numeric and handle missing values
    for_fcst_df = for_fcst_df.dropna()
    for_fcst_df = for_fcst_df.apply(pd.to_numeric, errors='coerce')
    
    # Remove any remaining rows with NaN values
    for_fcst_df = for_fcst_df.dropna()

    # Fit the model
    model = sm.OLS(for_fcst_df[series_name], sm.add_constant(for_fcst_df['t']), hasconst=True)
    results = model.fit()
    intercept = results.params['const']
    beta = results.params['t']
    return intercept, beta

# Convert DataFrame to dictionary
data_dict = df.to_dict(orient='index')

# Clean the data and create a list of cleaned data
cleaned_data_list = []
for k, v in data_dict.items():
    nested_list = [[sub_k, sub_v] for sub_k, sub_v in v.items() if sub_v != '\xa0' and pd.notnull(sub_v)]
    if nested_list:
        cleaned_data_list.append([k, nested_list])

# Iterate over the cleaned data list and compute intercept and beta
for i in range(len(cleaned_data_list)):
    sr = cleaned_data_list[i][0]
    for_fcst = for_forecast_df_from_orig_df(df, sr)
    intercept, beta = intercept_and_t_beta_from_for_forecast_df(for_fcst, sr)
    print(f"{sr} : {intercept + beta * 5}")

Net income : 74409.33333333328
Depreciation of property and equipment : 8551.99999999999
Stock-based compensation expense : 4897.999999999985
Deferred income taxes : 14463.333333333332
(Gain) loss on debt and equity securities, net : -28161.999999999996
Other : -1494.000000000001
Accounts receivable, net : -8938.999999999996
Income taxes, net : -2135.333333333333
Other assets : -2417.6666666666647
Accounts payable : -210.66666666666697
Accrued expenses and other liabilities : 11785.999999999995
Accrued revenue share : 2972.999999999999
Deferred revenue : 1053.3333333333328
Net cash provided by operating activities : 74776.33333333326
Purchases of property and equipment : -14236.666666666642
Purchases of marketable securities : -211985.33333333323
Maturities and sales of marketable securities : 187506.66666666657
Purchases of non-marketable securities : -2420.6666666666647
Maturities and sales of non-marketable securities : 650.9999999999995
Acquisitions, net of cash acquired, and purch